# Import module

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import json

In [59]:
# 載入自訂義檔案, 地址轉經緯度function
import geopyCoordinate as GC

# Cinema infomation webcraw

In [2]:
def dataset_webcraw(url):
    response = requests.get(url)
    res = BeautifulSoup(response.text) # 預設html.parser，不打也相同
    latest_data = res.find_all('div', class_ = 'download-item')[-1].a['href']
    return latest_data
url = 'https://data.gov.tw/dataset/22213'
latest_data = dataset_webcraw(url)

# Data Clean & Feature

In [ ]:
df = pd.read_csv(latest_data)
cinema_data = df[['事業名稱','地址']]
cinema_data.dropna(inplace=True)

In [ ]:
# 臺 轉 台
cinema_data['地址'] = cinema_data['地址'].apply(lambda x: '台' + x[1:] if x.startswith('臺') else x)

In [ ]:
# 新增經緯度，按下去就會刷google API 哦!
cinema_data['經緯度'] = cinema_data['地址'].map(GC.get_latitude_longitude)

In [ ]:
# 經緯度turple轉string
cinema_data['經緯度'] = cinema_data['經緯度'].map(json.dumps)

# Save to DataBase、JSON

In [28]:
# 不帶index 轉DB
conn = sqlite3.connect('TW_cinema_info.db')
cinema_data.to_sql('TW_cinema_info_latest', con=conn, if_exists='replace', index=False)
conn.close()

In [62]:
# 不index 轉JSON
cinema_data.to_json('TW_cinema_info_latest.json', orient='records', force_ascii=False)

# 載入DB、JSON

In [9]:
# JSON
with open('TW_cinema_info_latest.json', 'r', encoding='utf-8') as f:
    data_lst = json.load(f)

In [13]:
# DB
conn = sqlite3.connect('TW_cinema_info.db')
data_df = pd.read_sql_query("SELECT * FROM TW_cinema_info_latest", conn)
conn.close()